In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
dat = pd.read_excel('../data/Klaeger_Science_2017 Supplementary Table 2 Target Lists.xlsx', sheet_name='Kinobeads')
dat.head()

,Drug,Lysate,Beads,Gene Name,Relative Intensity DMSO,Relative Intensity 3 nM,Relative Intensity 10 nM,Relative Intensity 30 nM,Relative Intensity 100 nM,Relative Intensity 300 nM,...,Bottom,Top,Inflection,EC50,EC50 Standard Error,Correction Factor,Apparent Kd,R2,BIC,Target Classification
0,Abemaciclib,4 cell line mix,Kinobeads,AAK1,1,0.905556,0.791054,0.738292,0.639565,0.323354,...,0.009092,0.957905,151.288760,151.288760,45.696999,0.673073,101.828368,0.985386,-20.503519,High confidence
1,Abemaciclib,4 cell line mix,Kinobeads,ADCK1,1,1.045974,0.869866,1.141565,0.000000,0.000000,...,-0.001200,1.014355,63.935536,63.935536,68.832970,0.622595,39.805937,0.983520,-12.606135,High confidence
2,Abemaciclib,4 cell line mix,Kinobeads,BMP2K,1,1.101696,1.012297,1.087891,0.973740,0.863230,...,-1.003278,1.034558,84774.108960,84774.108960,217756.243215,0.660952,56031.642989,0.937247,-15.330485,High confidence
3,Abemaciclib,4 cell line mix,Kinobeads,CAMK2D,1,0.894551,0.984481,0.900583,0.440384,0.178266,...,0.018690,0.972850,95.373379,95.373379,12.131433,0.203913,19.447883,0.992088,-22.486948,High confidence
4,Abemaciclib,4 cell line mix,Kinobeads,CAMK2G,1,1.125435,1.565245,1.236451,0.315507,0.139777,...,0.043265,1.232332,82.363844,82.363844,NaN,0.217204,17.889794,0.939239,1.734623,High confidence


In [4]:
cols_to_keep = ['Drug', 'Gene Name', 'R2', 'Target Classification']
dat = dat[cols_to_keep]
dat.head()

,Drug,Gene Name,R2,Target Classification
0,Abemaciclib,AAK1,0.985386,High confidence
1,Abemaciclib,ADCK1,0.983520,High confidence
2,Abemaciclib,BMP2K,0.937247,High confidence
3,Abemaciclib,CAMK2D,0.992088,High confidence
4,Abemaciclib,CAMK2G,0.939239,High confidence


In [23]:
double_drug_targets = dat[dat['Gene Name'].apply(lambda x: ';' in x)]
double_drug_targets.shape

(128, 4)

In [34]:
first_genes = double_drug_targets['Gene Name'].apply(lambda x: x.split(';')[0])
second_genes = double_drug_targets['Gene Name'].apply(lambda x: x.split(';')[1])

repaired_double_targets = double_drug_targets.drop('Gene Name', axis=1).assign(**{'Gene Name':first_genes})
repaired_double_targets = repaired_double_targets.append(double_drug_targets.drop('Gene Name', axis=1).assign(**{'Gene Name':second_genes})) 
repaired_double_targets = repaired_double_targets[cols_to_keep]

In [37]:
dat = dat.drop(double_drug_targets.index).append(repaired_double_targets)
dat.head()

,Drug,Gene Name,R2,Target Classification
0,Abemaciclib,AAK1,0.985386,High confidence
1,Abemaciclib,ADCK1,0.983520,High confidence
2,Abemaciclib,BMP2K,0.937247,High confidence
3,Abemaciclib,CAMK2D,0.992088,High confidence
4,Abemaciclib,CAMK2G,0.939239,High confidence


In [38]:
genes_targeted = pd.unique(dat['Gene Name'])
len(genes_targeted)

531

In [39]:
louvain_tier1 = pd.read_csv('../results/louvain_clusters.txt', sep='\t')
len(set(louvain_tier1['names']) & set(genes_targeted))

238

In [12]:
alias = pd.read_excel('../data/KINASESmasterlist_w_Aliases.xlsx')
alias.head()

,Uniprot Protein,MS Gene,RNAseq Gene,RNAseq Accession,Family,Mouse Uniprot Protein,Mouse RNAseq gene,Mouse RNAseq Accession,Kinome Render Tree Name,Aliases (Conservative),...,Aliases,description,other_designations,Entrez_Symbol,Old_Name,Entrez_Synonyms,Entrez_other_designations,Unnamed: 18,Gene Symbol,Gene Symbol and Synonyms
0,AAK1,AAK1,AAK1,22848.0,Other,Aak1,Aak1,269774,AAK1,"AAK1,",...,NaN,AP2 associated kinase 1,adaptor-associated kinase 1,AAK1,AAK1,KIAA1048|MGC138170,adaptor-associated kinase 1,NaN,AAK1,AAK1
1,AAPK1,PRKAA1,PRKAA1,5562.0,CAMK,Aapk1,Prkaa1,105787,AMPK[alpha]1,"PRKAA1, AMPKa1",...,"AMPK, AMPKa1","protein kinase, AMP-activated, alpha 1 catalyt...","5'-AMP-activated protein kinase, catalytic alp...",PRKAA1,AMPKa1,AMPK|AMPKa1|MGC33776|MGC57364,"5'-AMP-activated protein kinase, catalytic alp...",NaN,PRKAA1,"AMPK, PRKAA1, AMPKa1"
2,AAPK2,PRKAA2,PRKAA2,5563.0,CAMK,Aapk2,Prkaa2,108079,AMPK[alpha]2,"PRKAA2, AMPK2, AMPKa2, PRKAA",...,"AMPK, AMPK2, AMPKa2, PRKAA","protein kinase, AMP-activated, alpha 2 catalyt...","5'-AMP-activated protein kinase, catalytic alp...",PRKAA2,AMPKa2,AMPK|AMPK2|PRKAA,"5'-AMP-activated protein kinase, catalytic alp...",NaN,PRKAA2,"AMPK2, AMPK, PRKAA, AMPKa2, PRKAA2"
3,ABL1,ABL1,ABL1,25.0,TK,Abl1,Abl1,11350,Abl,"ABL1,ABL, JTK7, bcr/abl, c-ABL, c-ABL1, v-abl",...,"ABL, JTK7, bcr/abl, c-ABL, c-ABL1, p150, v-abl","ABL proto-oncogene 1, non-receptor tyrosine ki...",Abelson tyrosine-protein kinase 1|bcr/c-abl on...,ABL1,ABL,ABL|JTK7|c-ABL|p150|v-abl,Abelson murine leukemia viral (v-abl) oncogene...,NaN,ABL1,"c-ABL1, p150, ABL, JTK7, v-abl, bcr/abl, c-ABL..."
4,ABL2,ABL2,ABL2,27.0,TK,Abl2,Abl2,11352,Arg,"ABL2,ABLL, ARG",...,"ABLL, ARG","ABL proto-oncogene 2, non-receptor tyrosine ki...","abelson-related gene protein|c-abl oncogene 2,...",ABL2,ARG,ABLL|ARG,Abelson murine leukemia viral (v-abl) oncogene...,NaN,ABL2,"ABL2, ABLL, ARG"


In [43]:
best_col = 'Uniprot Protein'
best_num = len(set(alias[best_col]) & set(genes_targeted))
for c in alias.columns:
    temp_set = set(alias[c])
    temp_len = len(set(alias[c]) & set(genes_targeted))
    if temp_len > best_num:
        best_col = c
        best_num = temp_len
        
print(best_col, best_num)

MS Gene 249


In [41]:
best_col = 'Uniprot Protein'
best_num = len(set(alias[best_col]) & set(louvain_tier1['names']))
for c in alias.columns:
    temp_set = set(alias[c])
    temp_len = len(set(alias[c]) & set(louvain_tier1['names']))
    if temp_len > best_num:
        best_col = c
        best_num = temp_len
        
print(best_col, best_num)

RNAseq Gene 473


### Alias note

RNAseq Gene is the column we use for our clusterings, while MS Gene appears to be the column used by Klaeger 2017

In [52]:
from collections import Counter

MS_to_RNAseq_mapper = alias.set_index('MS Gene')['RNAseq Gene'].to_dict()
targeted_kinases = list(map(lambda x: MS_to_RNAseq_mapper[x], list(set(alias[best_col]) & set(genes_targeted))))
len(targeted_kinases)

249

In [54]:
raw_counts_klaeger = Counter(dat['Gene Name'])

targeted_kinase_counts = {MS_to_RNAseq_mapper[gen]:count for gen,count in raw_counts_klaeger.items() if gen in targeted_kinases}

In [63]:
pd.Series(targeted_kinase_counts).to_csv('../results/targeted_kinase_counts.csv', sep=',')